<a href="https://colab.research.google.com/github/Meta-Sean/Practical-Deep-Learning/blob/main/corncomp_pt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle fastkaggle fastai timm>=0.6.2.dev0 

In [2]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
! mkdir ~/.kaggle #creating folder
! cp kaggle.json ~/.kaggle/ #copying kaggle.json
! chmod 600 ~/.kaggle/kaggle.json #reading the file with full access

In [4]:
from fastkaggle import * 
from fastai.vision.all import *

comp = 'kaggle-pog-series-s01e03'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

100%|██████████| 491M/491M [00:21<00:00, 23.5MB/s]


In [5]:
from pathlib import Path
path = Path('/content/kaggle-pog-series-s01e03/corn/')
path

Path('/content/kaggle-pog-series-s01e03/corn')

In [6]:
trn_path = path/'train'
files = get_image_files(trn_path)
tst_files = get_image_files(path/'test').sorted()

In [7]:
train_df = pd.read_csv(path/'train.csv')
train_df.head()

,seed_id,view,image,label
0,0,top,train/00000.png,broken
1,1,bottom,train/00001.png,pure
2,3,top,train/00003.png,broken
3,4,top,train/00004.png,pure
4,5,top,train/00005.png,discolored


In [8]:
def train(arch, size, item=Resize(132, method='squish'), accum=1, finetune=True, epochs=12):
    dls = ImageDataLoaders.from_csv(path, 'train.csv', fn_col=2, label_col=3, item_tfms=item,
                                   batch_tfms=aug_transforms(size=size, min_scale=0.75), bs=64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=[error_rate, accuracy]).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)


In [9]:
df = pd.read_csv(path/'train.csv')
df.label.value_counts()

pure          5837
broken        4554
discolored    2504
silkcut       1427
Name: label, dtype: int64

In [10]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [13]:
report_gpu()

GPU:0
no processes are running


In [12]:
! pip install pynvml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 4.4 MB/s 


In [14]:
train('convnext_small_in22k', 128, epochs=1, accum=2, finetune=False)
report_gpu()

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_small_22k_224.pth


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.391351,1.293090,0.537360,0.462640,01:33


GPU:0
process       2092 uses     3205.000 MB GPU memory


In [15]:
res = 132,128

In [17]:
models = {
    'convnext_large_in22k': {
        (Resize(res), (138,128)),
    }, 'vit_large_patch16_224': {
        (Resize(132, method='squish'), 128),
        (Resize(res), 128),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(132, method='squish'), 112),
        (Resize(res), 112),
    }, 'swin_large_patch4_window7_224': {
        (Resize(res), 128),
    }
}

In [18]:
trn_path = path/'train'

In [ ]:
tta_res = []

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train(arch, size, item=item, accum=2)) #, epochs=1))
        gc.collect()
        torch.cuda.empty_cache()

--- convnext_large_in22k
(138, 128)
Resize -- {'size': (128, 132), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}


Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_large_22k_224.pth


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.041473,0.836334,0.302025,0.697975,01:24


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.662537,0.594146,0.240223,0.759777,02:19
1,0.620413,0.644819,0.247207,0.752793,02:14


In [ ]:
save_pickle('tta_res.pkl', tta_res)

In [ ]:
tta_prs = first(zip(*tta_res))

In [ ]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

In [ ]:
dls = ImageDataLoaders.from_csv(path, 'train.csv', fn_col=2, label_col=3, item_tfms=Resize(132, method='squish'),
                                   batch_tfms=aug_transforms(size=128, min_scale=0.75))

In [ ]:
idxs = avg_pr.argmax(dim=1)
vocab = np.array(dls.vocab)
ss = pd.read_csv(path/'sample_submission.csv')
ss['label'] = vocab[idxs]
ss.to_csv('subm.csv', index=False)